<a href="https://colab.research.google.com/github/garkul/Transcriptomics.msu/blob/master/HW_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Домашнее задание 7</h1>
<h3> Гаркуль Лидия 4 курс ФББ</h3>

<b>Задание:</b><br>
1. Протестируйте написанный нами EM-алгоритм на устойчивость к параметрам изначальной инициализации.
2. Напишите NextFlow-пайплайн, который бы включал в себя определение экспрессий при помощи kallisto | bustools, а потом устранение дублетов при помощи DropletUtils.

###Скачиваем необходимые пакеты

In [ ]:
!pip3 install scanpy scrublet scikit-misc anndata2ri

import os
os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 3.8 MB/s 
     |████████████████████████████████| 8.8 MB 49.8 MB/s 
     |████████████████████████████████| 96 kB 5.3 MB/s 
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 9.4 MB 13.4 MB/s 
     |████████████████████████████████| 965 kB 45.9 MB/s 
     |████████████████████████████████| 295 kB 65.7 MB/s 
     |████████████████████████████████| 1.1 MB 50.6 MB/s 
     |████████████████████████████████| 647 kB 49.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=e89fa15f63c1162b75cbf62222187b8e3834a3f3d61f00586314160cb10ec2e4
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a8950e053725912e6a8c42c4a3a241410f6487b8402542ea
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3

In [ ]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.special import logsumexp

In [ ]:
sc.set_figure_params(dpi=150)
sns.set_style("ticks")

!wget https://cf.10xgenomics.com/samples/cell-exp/6.1.0/10k_PBMC_3p_nextgem_Chromium_X/10k_PBMC_3p_nextgem_Chromium_X_raw_feature_bc_matrix.h5
adata = sc.read_10x_h5("10k_PBMC_3p_nextgem_Chromium_X_raw_feature_bc_matrix.h5")
adata.var_names_make_unique()

--2022-12-16 20:50:46--  https://cf.10xgenomics.com/samples/cell-exp/6.1.0/10k_PBMC_3p_nextgem_Chromium_X/10k_PBMC_3p_nextgem_Chromium_X_raw_feature_bc_matrix.h5
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.0.173, 104.18.1.173, 2606:4700::6812:ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62760214 (60M) [binary/octet-stream]
Saving to: ‘10k_PBMC_3p_nextgem_Chromium_X_raw_feature_bc_matrix.h5’

10k_PBMC_3p_nextgem 100%[===================>]  59.85M  18.7MB/s    in 3.2s    

2022-12-16 20:50:50 (18.7 MB/s) - ‘10k_PBMC_3p_nextgem_Chromium_X_raw_feature_bc_matrix.h5’ saved [62760214/62760214]



/usr/local/lib/python3.8/dist-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [ ]:
!wget https://github.com/pachterlab/kallisto/releases/download/v0.46.0/kallisto_linux-v0.46.0.tar.gz
!tar -xf kallisto_linux-v0.46.0.tar.gz
!cp kallisto/kallisto /usr/local/bin/

!wget https://github.com/BUStools/bustools/releases/download/v0.39.3/bustools_linux-v0.39.3.tar.gz
!tar -xf bustools_linux-v0.39.3.tar.gz
!cp bustools/bustools /usr/local/bin/

###1. Протестируйте написанный нами EM-алгоритм на устойчивость к параметрам изначальной инициализации.

In [ ]:
#EM с семинара:
def E_step(X, params, draw=False):
  LogProb = np.log([
      stats.norm(params["mu_empty"], params["sigma_empty"]).pdf(X),
      stats.norm(params["mu_int"], params["sigma_int"]).pdf(X), 
      stats.norm(params["mu_real"], params["sigma_real"]).pdf(X)
  ])
  logPrior = np.log([params["phi_empty"], 1 - params["phi_real"] - params["phi_empty"], params["phi_real"]])
  logNum = logPrior + LogProb.T
  RealSoftMax = logsumexp(logNum, axis=1)
  probabilities = np.exp(logNum - np.array([RealSoftMax]).T)
  return RealSoftMax, probabilities

def M_step(X, params, probabilities):
  prob_empty = probabilities[:, 0]
  prob_int = probabilities[:, 1]
  prob_real = probabilities[:, 2]

  # Вычисление новых значений
  phi_empty = prob_empty.sum() / len(X)
  phi_real = prob_real.sum() / len(X)

  mu_empty = prob_empty.dot(X) / np.sum(prob_empty)
  mu_int = prob_int.dot(X) / np.sum(prob_int)
  mu_real = prob_real.dot(X) / np.sum(prob_real)

  sigma_empty = (np.sum(prob_empty * (X - mu_empty) ** 2) / np.sum(prob_empty)) ** 0.5
  sigma_int = (np.sum(prob_int * (X - mu_int) ** 2) / np.sum(prob_int)) ** 0.5
  sigma_real = (np.sum(prob_real * (X - mu_real) ** 2) / np.sum(prob_real)) ** 0.5

  return {
      "phi_empty" : phi_empty,
      "phi_real" : phi_real,
      "mu_empty" : mu_empty,
      "mu_int" : mu_int,
      "mu_real" : mu_real,
      "sigma_empty": sigma_empty,
      "sigma_int": sigma_int,
      "sigma_real": sigma_real
  }

  def run_EM(X, params):
    avg_allprob = []

    while True:
        allprob, probabilities = E_step(X, params)
        avg_allprob.append(np.mean(probabilities))
        if len(avg_allprob) > 2 and abs(avg_allprob[-1] - avg_allprob[-2]) < 0.00001:
            break
        params = M_step(X, params, probabilities)

    allprob, probabilities = E_step(X, params)
    forecasts = np.argmax(probabilities, axis=1)
    return forecasts, probabilities, avg_allprob

In [ ]:
#как и в семинаре, уберем клетки с количеством umi меньше 10. Создадим np.array X для тестирования
min_umi = 10
adata = adata[adata.X.sum(axis=1).A.T[0] >= min_umi]
X = np.log(adata.X.sum(axis=1).A.T[0])
X

array([3.8066626, 4.0253515, 2.9957323, ..., 3.1354942, 3.465736 ,
       3.218876 ], dtype=float32)

In [ ]:
#зададим несколько вариантов парамеров
expected_cells = 10000 / len(adata)

params = {
  "phi_real" : expected_cells,
  "phi_empty" : (1 - expected_cells) / 2,
  "mu_real" : np.log(10000),
  "mu_int" : np.log(500),
  "mu_empty" : np.log(10),
  "sigma_real" : 1,
  "sigma_int" : 1,
  "sigma_empty" : 1
}

params = {
  "phi_real" : 1,
  "phi_empty" : 1,
  "mu_real" : 1,
  "mu_int" : 1,
  "mu_empty" : 1,
  "sigma_real" : 1,
  "sigma_int" : 1,
  "sigma_empty" : 1
}

In [ ]:
pred, prob, avg_allprob = run_EM(X, params)
pred_1, prob_1, avg_allprob_1 = run_EM(X, params_1)

<ipython-input-25-e2ba2dd118a4>:8: RuntimeWarning: invalid value encountered in log
  logPrior = np.log([params["phi_empty"], 1 - params["phi_real"] - params["phi_empty"], params["phi_real"]])


###2. Напишите NextFlow-пайплайн, который бы включал в себя определение экспрессий при помощи kallisto | bustools, а потом устранение дублетов при помощи DropletUtils.




```
params.results_dir = "results/"
reads_list = params.reads.split(",")
params.index = "transcriptome.idx"

log.info ""
log.info " KALLISTO | BUSTOOLS  >  DROPLETUTILS "
log.info "================================="
log.info "reads number(s)         : ${reads_list}"
log.info "Results location   : ${params.results_dir}"
log.info "Index : ${params.index}"

process kallisto {
  publishDir "${params.results_dir}"

  input:
  path x

  output:
  path "results/*"

  script:
    """
    $ kb count -i ${params.index} -g ${params.t2g} -x ${params.version_chem} -t2 -o ${params.results_dir} ${read}
    """
}


process DropletUtils {
  publishDir "${params.results_dir}"

  input:
  path x

  output:
  path "results/*"

  script:
    """
    $ ro.r("library(DropletUtils)")
    $ ro.globalenv["adata_raw"] = adata
    $df = ro.r('e.out <- emptyDrops(assay(adata_raw))')
    """
}


workflow {
  data = Channel.of( reads_list )
  kallisto(data)
  DropletUtils(kallisto.out)
}
```

